<a href="https://colab.research.google.com/github/Kinrider/TechChallenge/blob/main/03%20-%20Scripts/ETL_exp_prod_venti_della_valle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importando Bibliotecas

In [135]:
import pandas as pd
import random as rd
import numpy as np

# Define as opções para exibir floats sem notação científica

pd.set_option('display.float_format', '{:.0f}'.format)

# Tratamento de dados de Exportação

In [136]:
# Para ler arquivos do Git, sempre específicar no final : '?raw=true'

dados = 'https://github.com/Kinrider/TechChallenge/blob/cad34f20bb46229c8cc8b78d6a21143aa68d6726/Fontes/ExpVinho.csv?raw=true'

In [137]:
vinhosexp = pd.read_csv(dados, sep=';' , encoding= 'utf8')
vinhosexp
x = vinhosexp.copy()

In [138]:
x.head()

,Id,País,1970,1970.1,1971,1971.1,1972,1972.1,1973,1973.1,...,2019,2019.1,2020,2020.1,2021,2021.1,2022,2022.1,2023,2023.1
0,1,Afeganistão,0,0,0,0,0,0,0,0,...,0,0,0,0,11,46,0,0,0,0
1,2,África do Sul,0,0,0,0,0,0,0,0,...,26,95,4,21,0,0,0,0,117,698
2,3,"Alemanha, República Democrática",0,0,0,0,4168,2630,12000,8250,...,3660,25467,6261,32605,2698,6741,7630,45367,4806,31853
3,4,Angola,0,0,0,0,0,0,0,0,...,345,1065,0,0,0,0,4068,4761,0,0
4,5,Anguilla,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [139]:
array_dollar=['Id' , 'País']
array_kilos=['Id' , 'País']

for column in x.columns:
  if '.1' in column:
   dollar = column.replace('.1' , '')+'_'+'dollar'
   x.rename({column:dollar},inplace=True, axis=1)
   array_dollar.append(dollar)
  else:
    kilos = column +'_'+ 'kilos'
    x.rename({column:kilos},inplace=True, axis=1)
    array_kilos.append(kilos)

In [140]:
x.rename({'Id_kilos' :'Id'},inplace=True, axis=1)
x.rename({'País_kilos':'País'},inplace=True, axis=1)

In [141]:
array_kilos.remove('Id_kilos')
array_kilos.remove('País_kilos')

In [142]:
x[array_kilos].head()

,Id,País,1970_kilos,1971_kilos,1972_kilos,1973_kilos,1974_kilos,1975_kilos,1976_kilos,1977_kilos,...,2014_kilos,2015_kilos,2016_kilos,2017_kilos,2018_kilos,2019_kilos,2020_kilos,2021_kilos,2022_kilos,2023_kilos
0,1,Afeganistão,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,11,0,0
1,2,África do Sul,0,0,0,0,0,0,0,0,...,0,0,0,0,0,26,4,0,0,117
2,3,"Alemanha, República Democrática",0,0,4168,12000,0,0,0,0,...,213348,10680,14012,15467,10794,3660,6261,2698,7630,4806
3,4,Angola,0,0,0,0,0,0,0,0,...,12182,1908,7359,10170,477,345,0,0,4068,0
4,5,Anguilla,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [143]:
x[array_dollar].head()

,Id,País,1970_dollar,1971_dollar,1972_dollar,1973_dollar,1974_dollar,1975_dollar,1976_dollar,1977_dollar,...,2014_dollar,2015_dollar,2016_dollar,2017_dollar,2018_dollar,2019_dollar,2020_dollar,2021_dollar,2022_dollar,2023_dollar
0,1,Afeganistão,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,46,0,0
1,2,África do Sul,0,0,0,0,0,0,0,0,...,0,0,0,0,0,95,21,0,0,698
2,3,"Alemanha, República Democrática",0,0,2630,8250,0,0,0,0,...,761653,44780,68109,87702,45382,25467,32605,6741,45367,31853
3,4,Angola,0,0,0,0,0,0,0,0,...,23124,17089,35390,61680,709,1065,0,0,4761,0
4,5,Anguilla,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [144]:
df = x

In [145]:
# Reshape do dataframe

# 1º transpor kilos para linha

df1 = df.melt(
    id_vars = ['Id', 'País'],
    value_vars = df.filter(like = '_kilos').columns,
    var_name = 'ano',
    value_name = 'kilos'
    )

# 2º transpor dólar para linha

df2 = df.melt(
    id_vars = ['Id', 'País'],
    value_vars = df.filter(like = '_dollar').columns,
    var_name = 'anos',
    value_name = 'dollar'
    )

# 3º sincronizar os índices

df1 = df1.set_index(['Id','País', df1.groupby(['Id','País']).cumcount()])
df2 = df2.set_index(['Id','País', df1.groupby(['Id','País']).cumcount()])

# 4º Juntar data_frames

df3 = (
    pd.concat([df1, df2], axis=1)
  .sort_index(level=2)
  .reset_index(level=2, drop=True)
  .reset_index()
  )

df3.head()

# referências:
# https://stackoverflow.com/questions/51519101/simultaneously-melt-multiple-columns-in-python-pandas
# https://pandas.pydata.org/docs/reference/api/pandas.melt.html
# https://medium.com/horadecodar/data-science-tips-02-como-usar-loc-e-iloc-no-pandas-fab58e214d87

,Id,País,ano,kilos,anos,dollar
0,1,Afeganistão,1970_kilos,0,1970_dollar,0
1,2,África do Sul,1970_kilos,0,1970_dollar,0
2,3,"Alemanha, República Democrática",1970_kilos,0,1970_dollar,0
3,4,Angola,1970_kilos,0,1970_dollar,0
4,5,Anguilla,1970_kilos,0,1970_dollar,0


In [146]:
# Dropa coluna de ano duplicada

df3.drop(['anos'], axis=1, inplace = True)

In [147]:
# tratando o campo de ano

df3['ano'] = df3['ano'].str.replace('_kilos','') + '-01-01'

df3.head()

,Id,País,ano,kilos,dollar
0,1,Afeganistão,1970-01-01,0,0
1,2,África do Sul,1970-01-01,0,0
2,3,"Alemanha, República Democrática",1970-01-01,0,0
3,4,Angola,1970-01-01,0,0
4,5,Anguilla,1970-01-01,0,0


In [148]:
# tranformando string em data

df3['ano'] = pd.to_datetime(df3['ano'],format='%Y-%m-%d')

df3.head()

,Id,País,ano,kilos,dollar
0,1,Afeganistão,1970-01-01,0,0
1,2,África do Sul,1970-01-01,0,0
2,3,"Alemanha, República Democrática",1970-01-01,0,0
3,4,Angola,1970-01-01,0,0
4,5,Anguilla,1970-01-01,0,0


In [149]:
# fitrando dados de 2008 pra frente:

df3.query('ano >= "2008-01-01"' ,inplace = True)

In [150]:
df3.head()

,Id,País,ano,kilos,dollar
5206,1,Afeganistão,2008-01-01,0,0
5207,2,África do Sul,2008-01-01,0,0
5208,3,"Alemanha, República Democrática",2008-01-01,265742,429970
5209,4,Angola,2008-01-01,25721,71083
5210,5,Anguilla,2008-01-01,0,0


In [151]:
# Verificando nomenclatura dos países

df3['País'].unique()

array(['Afeganistão', 'África do Sul', 'Alemanha, República Democrática',
       'Angola', 'Anguilla', 'Antígua e Barbuda', 'Antilhas Holandesas',
       'Arábia Saudita', 'Argentina', 'Aruba', 'Austrália', 'Áustria',
       'Bahamas', 'Bangladesh', 'Barbados', 'Barein', 'Bélgica', 'Belice',
       'Benin', 'Bermudas', 'Bolívia', 'Bósnia-Herzegovina', 'Brasil',
       'Bulgária', 'Cabo Verde', 'Camarões', 'Canadá', 'Catar',
       'Cayman, Ilhas', 'Chile', 'China', 'Chipre', 'Cingapura',
       'Cocos (Keeling), Ilhas', 'Colômbia', 'Comores', 'Congo',
       'Coreia, Republica Sul', 'Costa do Marfim', 'Costa Rica',
       'Croácia', 'Cuba', 'Curaçao', 'Dinamarca', 'Dominica',
       'El Salvador', 'Emirados Arabes Unidos', 'Equador',
       'Eslovaca, Republica', 'Espanha', 'Estados Unidos', 'Estônia',
       'Filipinas', 'Finlândia', 'França', 'Gana', 'Gibraltar', 'Granada',
       'Grécia', 'Guatemala', 'Guiana', 'Guiana Francesa', 'Guine Bissau',
       'Guine Equatorial', 'Haiti', 

In [152]:
# Solicitado ao GPT que gerasse a lista de de_para, considerando os países listados acima

lista_paises = [
    ('Afeganistão', 'Afghanistan', 'AFG'),
    ('África do Sul', 'South Africa', 'ZAF'),
    ('Alemanha, República Democrática', 'Germany', 'DEU'),
    ('Angola', 'Angola', 'AGO'),
    ('Anguilla', 'Anguilla', 'AIA'),
    ('Antígua e Barbuda', 'Antigua and Barbuda', 'ATG'),
    ('Antilhas Holandesas', 'Netherlands Antilles', 'ANT'),
    ('Arábia Saudita', 'Saudi Arabia', 'SAU'),
    ('Argentina', 'Argentina', 'ARG'),
    ('Aruba', 'Aruba', 'ABW'),
    ('Austrália', 'Australia', 'AUS'),
    ('Áustria', 'Austria', 'AUT'),
    ('Bahamas', 'Bahamas', 'BHS'),
    ('Bangladesh', 'Bangladesh', 'BGD'),
    ('Barbados', 'Barbados', 'BRB'),
    ('Barein', 'Bahrain', 'BHR'),
    ('Bélgica', 'Belgium', 'BEL'),
    ('Belice', 'Belize', 'BLZ'),
    ('Benin', 'Benin', 'BEN'),
    ('Bermudas', 'Bermuda', 'BMU'),
    ('Bolívia', 'Bolivia', 'BOL'),
    ('Bósnia-Herzegovina', 'Bosnia and Herzegovina', 'BIH'),
    ('Brasil', 'Brazil', 'BRA'),
    ('Bulgária', 'Bulgaria', 'BGR'),
    ('Cabo Verde', 'Cabo Verde', 'CPV'),
    ('Camarões', 'Cameroon', 'CMR'),
    ('Canadá', 'Canada', 'CAN'),
    ('Catar', 'Qatar', 'QAT'),
    ('Cayman, Ilhas', 'Cayman Islands', 'CYM'),
    ('Chile', 'Chile', 'CHL'),
    ('China', 'China, mainland', 'CHN'),
    ('Chipre', 'Cyprus', 'CYP'),
    ('Cingapura', 'Singapore', 'SGP'),
    ('Cocos (Keeling), Ilhas', 'Cocos (Keeling) Islands', 'CCK'),
    ('Colômbia', 'Colombia', 'COL'),
    ('Comores', 'Comoros', 'COM'),
    ('Congo', 'Congo', 'COG'),
    ('Coreia, Republica Sul', 'Korea (the Republic of)', 'KOR'),
    ('Costa do Marfim', "Côte d'Ivoire", 'CIV'),
    ('Costa Rica', 'Costa Rica', 'CRI'),
    ('Croácia', 'Croatia', 'HRV'),
    ('Cuba', 'Cuba', 'CUB'),
    ('Curaçao', 'Curacao', 'CUW'),
    ('Dinamarca', 'Denmark', 'DNK'),
    ('Dominica', 'Dominica', 'DMA'),
    ('El Salvador', 'El Salvador', 'SLV'),
    ('Emirados Arabes Unidos', 'United Arab Emirates', 'ARE'),
    ('Equador', 'Ecuador', 'ECU'),
    ('Eslovaca, Republica', 'Slovakia', 'SVK'),
    ('Espanha', 'Spain', 'ESP'),
    ('Estados Unidos', 'United States of America', 'USA'),
    ('Estônia', 'Estonia', 'EST'),
    ('Filipinas', 'Philippines', 'PHL'),
    ('Finlândia', 'Finland', 'FIN'),
    ('França', 'France', 'FRA'),
    ('Gana', 'Ghana', 'GHA'),
    ('Gibraltar', 'Gibraltar', 'GIB'),
    ('Granada', 'Grenada', 'GRD'),
    ('Grécia', 'Greece', 'GRC'),
    ('Guatemala', 'Guatemala', 'GTM'),
    ('Guiana', 'Guyana', 'GUY'),
    ('Guiana Francesa', 'French Guiana', 'GUF'),
    ('Guine Bissau', 'Guinea-Bissau', 'GNB'),
    ('Guine Equatorial', 'Equatorial Guinea', 'GNQ'),
    ('Haiti', 'Haiti', 'HTI'),
    ('Honduras', 'Honduras', 'HND'),
    ('Hong Kong', 'China, Hong Kong SAR', 'HKG'),
    ('Hungria', 'Hungary', 'HUN'),
    ('Ilha de Man', 'Isle of Man', 'IMN'),
    ('Ilhas Virgens', 'Virgin Islands', 'VIR'),
    ('India', 'India', 'IND'),
    ('Indonésia', 'Indonesia', 'IDN'),
    ('Irã', 'Iran', 'IRN'),
    ('Iraque', 'Iraq', 'IRQ'),
    ('Irlanda', 'Ireland', 'IRL'),
    ('Itália', 'Italy', 'ITA'),
    ('Jamaica', 'Jamaica', 'JAM'),
    ('Japão', 'Japan', 'JPN'),
    ('Jordânia', 'Jordan', 'JOR'),
    ('Letônia', 'Latvia', 'LVA'),
    ('Líbano', 'Lebanon', 'LBN'),
    ('Libéria', 'Liberia', 'LBR'),
    ('Luxemburgo', 'Luxemburg', 'LUX'),
    ('Macau', 'China, Macao SAR', 'MAC'),
    ('Malásia', 'Malaysia', 'MYS'),
    ('Malavi', 'Malawi', 'MWI'),
    ('Malta', 'Malta', 'MLT'),
    ('Marshall, Ilhas', 'Marshall Islands', 'MHL'),
    ('Martinica', 'Martinique', 'MTQ'),
    ('Mauritânia', 'Mauritania', 'MRT'),
    ('México', 'Mexico', 'MEX'),
    ('Moçambique', 'Mozambique', 'MOZ'),
    ('Montenegro', 'Montenegro', 'MNE'),
    ('Namíbia', 'Namibia', 'NAM'),
    ('Nicarágua', 'Nicaragua', 'NIC'),
    ('Nigéria', 'Nigeria', 'NGA'),
    ('Noruega', 'Norway', 'NOR'),
    ('Nova Caledônia', 'New Caledonia', 'NCL'),
    ('Nova Zelândia', 'New Zealand', 'NZL'),
    ('Omã', 'Oman', 'OMN'),
    ('Países Baixos', 'Netherlands', 'NLD'),
    ('Palau', 'Palau', 'PLW'),
    ('Panamá', 'Panama', 'PAN'),
    ('Paraguai', 'Paraguay', 'PRY'),
    ('Peru', 'Peru', 'PER'),
    ('Pitcairn', 'Pitcairn Islands', 'PCN'),
    ('Polônia', 'Poland', 'POL'),
    ('Porto Rico', 'Puerto Rico', 'PRI'),
    ('Portugal', 'Portugal', 'PRT'),
    ('Quênia', 'Kenya', 'KEN'),
    ('Reino Unido', 'United Kingdom', 'GBR'),
    ('República Dominicana', 'Dominican Republic', 'DOM'),
    ('Rússia', 'Russia', 'RUS'),
    ('São Cristóvão e Névis', 'Saint Kitts and Nevis', 'KNA'),
    ('São Tomé e Príncipe', 'Sao Tome and Principe', 'STP'),
    ('São Vicente e Granadinas', 'Saint Vincent and the Grenadines', 'VCT'),
    ('Senegal', 'Senegal', 'SEN'),
    ('Serra Leoa', 'Sierra Leone', 'SLE'),
    ('Singapura', 'Singapore', 'SGP'),
    ('Suazilândia', 'Eswatini', 'SWZ'),
    ('Suécia', 'Sweden', 'SWE'),
    ('Suíça', 'Switzerland', 'CHE'),
    ('Suriname', 'Suriname', 'SUR'),
    ('Tailândia', 'Thailand', 'THA'),
    ('Taiwan (Formosa)', 'China, Taiwan Province of', 'TWN'),
    ('Tanzânia', 'Tanzania, the United Republic of', 'TZA'),
    ('Tcheca, República', 'Czech Republic', 'CZE'),
    ('Togo', 'Togo', 'TGO'),
    ('Toquelau', 'Tokelau', 'TKL'),
    ('Trinidade Tobago', 'Trinidad and Tobago', 'TTO'),
    ('Tunísia', 'Tunisia', 'TUN'),
    ('Turquia', 'Republic of Türkiye', 'TUR'),
    ('Tuvalu', 'Tuvalu', 'TUV'),
    ('Uruguai', 'Uruguay', 'URY'),
    ('Vanuatu', 'Vanuatu', 'VUT'),
    ('Venezuela', 'Venezuela', 'VEN'),
    ('Vietnã', 'Vietnam', 'VNM')
]

# Tranformando a tupla de países em um dataframe:

de_para_pais = pd.DataFrame(lista_paises, columns = ['pais_pt','pais_in','cod_pais'])

In [153]:
# join com de_para de paises

df_exp = df3.merge(de_para_pais, how = 'left', left_on=['País'],right_on = ['pais_pt'])

In [154]:
# Dropa colunas não utilizadas:

df_exp.drop(['País','pais_in','Id'], axis=1, inplace = True)

In [155]:
df_exp.head()

,ano,kilos,dollar,pais_pt,cod_pais
0,2008-01-01,0,0,Afeganistão,AFG
1,2008-01-01,0,0,África do Sul,ZAF
2,2008-01-01,265742,429970,"Alemanha, República Democrática",DEU
3,2008-01-01,25721,71083,Angola,AGO
4,2008-01-01,0,0,Anguilla,AIA


#Estimando Exportacao Venti della valle

In [156]:
df_exp.head()

,ano,kilos,dollar,pais_pt,cod_pais
0,2008-01-01,0,0,Afeganistão,AFG
1,2008-01-01,0,0,África do Sul,ZAF
2,2008-01-01,265742,429970,"Alemanha, República Democrática",DEU
3,2008-01-01,25721,71083,Angola,AGO
4,2008-01-01,0,0,Anguilla,AIA


In [157]:
# Define uma função que vai aplicar sobre o total um percentual entre 18% a 24% do volume total para a nossa vinícola

def mult(x):
  return (rd.randint(18, 24)/100) * x

In [158]:
# Reinicia o índice para futura iteração

df_exp.reset_index(drop=True)

,ano,kilos,dollar,pais_pt,cod_pais
0,2008-01-01,0,0,Afeganistão,AFG
1,2008-01-01,0,0,África do Sul,ZAF
2,2008-01-01,265742,429970,"Alemanha, República Democrática",DEU
3,2008-01-01,25721,71083,Angola,AGO
4,2008-01-01,0,0,Anguilla,AIA
...,...,...,...,...,...
2187,2023-01-01,0,0,Tuvalu,TUV
2188,2023-01-01,326093,454271,Uruguai,URY
2189,2023-01-01,0,0,Vanuatu,VUT
2190,2023-01-01,141030,220512,Venezuela,VEN


In [159]:
#Itera sobre o index, de forma a aplicar para cada index na coluna criada chamada exportacao a função de multiplicação

for index, row in df_exp.iterrows():
  # Atualiza a célula da linha atual (index) e coluna 'producao' usando loc
  df_exp.loc[index, 'exportacao'] = mult(row['kilos'])

In [160]:
df_exp

,ano,kilos,dollar,pais_pt,cod_pais,exportacao
0,2008-01-01,0,0,Afeganistão,AFG,0
1,2008-01-01,0,0,África do Sul,ZAF,0
2,2008-01-01,265742,429970,"Alemanha, República Democrática",DEU,58463
3,2008-01-01,25721,71083,Angola,AGO,6173
4,2008-01-01,0,0,Anguilla,AIA,0
...,...,...,...,...,...,...
2187,2023-01-01,0,0,Tuvalu,TUV,0
2188,2023-01-01,326093,454271,Uruguai,URY,71740
2189,2023-01-01,0,0,Vanuatu,VUT,0
2190,2023-01-01,141030,220512,Venezuela,VEN,31027


In [161]:
#dropa colunas não utilizada

df_exp.drop(['kilos','dollar'], axis = 1, inplace = True)

# Estimando a Produção da Venti della Valle

In [162]:
df_prod = df_exp.copy()

In [163]:
# Mantém apenas colunas necessárias

df_prod = df_prod.groupby('ano')['exportacao'].sum()

# Tranforma novamente em dataframe

df_prod = df_prod.reset_index()

In [164]:
df_prod.head()

,ano,exportacao
0,2008-01-01,2183715
1,2009-01-01,6014245
2,2010-01-01,268444
3,2011-01-01,254342
4,2012-01-01,1255229


In [165]:
''' Temos o quanto exportamos, aleatoriamente aplicamos um percentual entre 37% a 42% representando o que é exportado
 a diferença é a nossa produção'''

def prod(x):
  return (1 - (rd.randint(37, 42)/100)) * x

In [166]:
#Itera sobre o index, de forma a aplicar para cada index na coluna criada chamada produção a função de prod

for index, row in df_prod.iterrows():
  # Atualiza a célula da linha atual (index) e coluna 'producao' usando loc
  df_prod.loc[index, 'producao'] = prod(row['exportacao'])

In [167]:
df_prod.head()

,ano,exportacao,producao
0,2008-01-01,2183715,1375740
1,2009-01-01,6014245,3488262
2,2010-01-01,268444,161066
3,2011-01-01,254342,157692
4,2012-01-01,1255229,765690


In [168]:
#dropa colunas não utilizada

df_prod.drop(['exportacao'], axis = 1, inplace = True)

In [169]:
pd.DataFrame(df_prod)

,ano,producao
0,2008-01-01,1375740
1,2009-01-01,3488262
2,2010-01-01,161066
3,2011-01-01,157692
4,2012-01-01,765690
5,2013-01-01,1010452
6,2014-01-01,299417
7,2015-01-01,152625
8,2016-01-01,214326
9,2017-01-01,352347


In [170]:
# Proporções base para os diferentes tipos de vinho
base_proporcoes = np.array([0.4, 0.3, 0.2, 0.1])  # Vinho tinto, branco, rosé, espumantes

# Definindo uma função para gerar proporções com pequena variação
def gerar_proporcoes(base, variacao = (rd.uniform(1.5, 2.2)/100)):
    variacao_aleatoria = np.random.uniform(-variacao, variacao, size=base.shape)
    nova_proporcao = base + variacao_aleatoria
    nova_proporcao = nova_proporcao / nova_proporcao.sum()  # Normaliza para que a soma seja 1
    return nova_proporcao

In [171]:
# Criando colunas para as categorias de vinho
for i, row in df_prod.iterrows():
    proporcoes = gerar_proporcoes(base_proporcoes)
    df_prod.loc[i, 'vinho_tinto'] = row['producao'] * proporcoes[0]
    df_prod.loc[i, 'vinho_branco'] = row['producao'] * proporcoes[1]
    df_prod.loc[i, 'vinho_rose'] = row['producao'] * proporcoes[2]
    df_prod.loc[i, 'espumantes'] = row['producao'] * proporcoes[3]
else:
  print("Coluna 'producao' não encontrada!")

Coluna 'producao' não encontrada!


In [172]:
# Dropa colunas não utilziadas

df_prod.drop(['producao'], axis = 1, inplace = True)

In [173]:
df_prod.head()

,ano,vinho_tinto,vinho_branco,vinho_rose,espumantes
0,2008-01-01,559711,392797,265995,157237
1,2009-01-01,1401239,1083898,642966,360159
2,2010-01-01,65036,48941,32208,14881
3,2011-01-01,62060,46441,31655,17537
4,2012-01-01,307044,237021,142937,78687


In [174]:
# Transposição

dfx1 = df_prod.melt(
    id_vars = ['ano'],
    #value_vars = ['vinho_tinto'],
    var_name = 'tipo_vinho',
    value_name = 'producao_litros'
    )



In [175]:
dfx1.query('ano == "2008-01-01"')

<ipython-input-175-345541b13775>:1: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  dfx1.query('ano == "2008-01-01"')


,ano,tipo_vinho,producao_litros
0,2008-01-01,vinho_tinto,559711
16,2008-01-01,vinho_branco,392797
32,2008-01-01,vinho_rose,265995
48,2008-01-01,espumantes,157237


# Salva Arquivos Gerados

In [176]:
##Salva Arquivo

#saída dados de exportacao
df_exp.to_excel('/content/drive/Othercomputers/Meu laptop/Fiap - Scripts/01 - Tech_Challenge/01 - Dados/exportacao_total_venti_della_valle.xlsx', index = False)

#saída dados de producao
df_exp.to_excel('/content/drive/Othercomputers/Meu laptop/Fiap - Scripts/01 - Tech_Challenge/01 - Dados/producao_venti_della_valle.xlsx', index = False)